In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,optimizers,datasets
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import tensorflow_datasets as tfds

C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data,info = tfds.load('horses_or_humans', split='train+test', shuffle_files=True, with_info=True,as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...:  20%|█████████████▋                                                     | 31/152 [00:24<01:43,  1.17 MiB/s]


Dl Size...:  42%|████████████████████████████▏                                      | 64/152 [00:54<02:01,  1.38s/ MiB]


Dl Size...:  64%|██████████████████████████████████████████▊                        | 97/152 [01:18<01:21,  1.48s/ MiB]


Dl Size...:  83%|██████████████████████████████████████████████████████▋           | 126/152 [02:10<00:35,  1.36s/ MiB]

In [5]:
vis = tfds.visualization.show_examples(data,info)

ValueError: Visualisation not supported for dataset `groove`